In [ ]:
# load MDS test dataset, get summaries for clusters, evaluate

In [1]:
import json
from pathlib import Path
import tqdm

from transformers import (modeling_utils,
                          BartTokenizer,
                          BartForConditionalGeneration,
                          BartConfig)

from transformer_decoding import decoding_utils
from transformer_decoding.test_decoding import get_start_state


In [2]:
bart_cnndm_model = BartForConditionalGeneration.from_pretrained('bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')

In [3]:
# see ``examples/summarization/bart/evaluate_cnn.py`` for a longer example

def article_to_text(article):
    return f'{article["title"]} {article["text"]}'


def summarize(args):

    model = args['model']
    tokenizer = args['tokenizer']
    decoding_hyperparams = {
        'max_length': args['max_length'],
        'num_beams': args['num_beams']
    }
    dataset = [json.loads(l) for l in open(args['evaluation_dataset'])]

    summaries = []
    # get summary for each cluster
    for cluster in tqdm.tqdm(dataset):
        articles = [article_to_text(a) for a in cluster['articles'][:args['max_articles']]]
        
        component_states = [get_start_state(a, model, tokenizer, decoding_hyperparams)
                            for a in articles]
        ensemble_state = get_start_state(articles[0], model, tokenizer, decoding_hyperparams)

        component_states, ensemble_state = \
            decoding_utils.generate(component_states, decoding_hyperparams['max_length'],
                                    ensemble_state=ensemble_state)
        
        #assert len(ensemble_state['input_ids']) == 1, 'We currently have batch size=1 (we decode one cluster at a time)'
        print(f'input_ids shape: {ensemble_state["input_ids"].shape}')
        print(f'Reference Summary:\n{cluster["summary"]}')
        predictions = [tokenizer.decode(input_ids,
                                        skip_special_tokens=True,
                                        clean_up_tokenization_spaces=False)
                       for input_ids in ensemble_state['input_ids']]
        print(f'Predictions: \n{predictions}')
        print()
        
        # NOTE: hack to just take the first one right now
        summaries.append((predictions[0], cluster['summary']))
    return summaries


In [4]:
evaluation_args = {
    'evaluation_dataset': '/home/chris/projects/aylien/dynamic-ensembles/data/WCEP/test.jsonl',
    'model': bart_cnndm_model,
    'tokenizer': bart_tokenizer,
    'max_length': 60,
    'num_beams': 1,
    'max_articles': 20
}

cluster_summaries = summarize(evaluation_args)

  0%|          | 1/1022 [01:01<17:24:02, 61.35s/it]

input_ids shape: torch.Size([1, 59])
Reference Summary:
A blast near the 11th-century Data Darbar Sufi shrine in Lahore, Pakistan, kills at least eleven people, half of them police officers.
Predictions: 
["The blast occurred in the city of Lahore, Pakistan. The blast occurred near a popular shrine. The attack was in the early hours of Wednesday morning. The death toll is expected to rise. The explosion occurred near the city's famous Sufi shrine. It was the second"]



  0%|          | 2/1022 [02:03<17:28:26, 61.67s/it]

input_ids shape: torch.Size([1, 56])
Reference Summary:
Nine people were killed after the Taliban launches an attack on the Kabul offices of international non-governmental organization Counterpart International.
Predictions: 
['The attack on the aid group is the latest in a series of attacks on Afghan forces. The attack comes as the Taliban continue to hold out against the U.S. government. The Taliban have been fighting a coalition government in Afghanistan for more than a decade.']



  0%|          | 3/1022 [03:06<17:30:47, 61.87s/it]

...

input_ids shape: torch.Size([1, 59])
Reference Summary:
New Zealand bans tourists from swimming with bottlenose dolphins, saying dwindling numbers are linked to excessive interaction with tourists, as the animals choose socialising with people over necessary biological functions. Authorities say the species risks "being loved into extinction".
Predictions: 
["New Zealand bans tourists from swimming with bottlenose dolphins. The move is in a bid to protect the species from being 'loved' by tourists. The population of bottlenoses has been declining in recent years. The dolphins are thought to be in danger of being wiped out."]



100%|█████████▉| 1021/1022 [15:05:17<00:38, 38.18s/it]

input_ids shape: torch.Size([1, 59])
Reference Summary:
28 people are killed in a bar fire in Coatzacoalcos, Veracruz, Mexico, that police believe was caused by an attack by drug cartels. Mexican media reports that Molotov cocktails were thrown inside.
Predictions: 
["The attack happened in the city of Coatzacoalcos, in Mexico's Gulf coast city of Mexico. The attack killed 23 people and injured 13 others. The attackers were believed to be from the local community. The victims were killed in a bar attack in the southern city"]



100%|██████████| 1022/1022 [15:05:46<00:00, 35.39s/it]

input_ids shape: torch.Size([1, 59])
Reference Summary:
Pretoria, the administrative capital of South Africa, experiences widespread looting and violent attacks by rioters focused on foreigners. Police are currently outnumbered and many businesses have been set on fire. The riots follow the death of a taxi driver reportedly shot by Nigerian drug dealers after they realized he had seen them complete a drug deal and had also seen their supplier. Bus services are suspended and authorities say the capital is currently unsafe.
Predictions: 
['The incident happened in the Pretoria CBD on Tuesday. The incident occurred after a taxi driver was killed in a road accident. The driver was allegedly shot by police after he tried to drive away from a road. The accident happened after a car was allegedly hit by a stray bullet']



In [5]:
import argparse
import collections
import numpy as np
# import utils
from newsroom.analyze.rouge import ROUGE_L, ROUGE_N


def print_mean(results, rouge_types):
    for rouge_type in rouge_types:
        precs = results[rouge_type]['p']
        recalls = results[rouge_type]['r']
        fscores = results[rouge_type]['f']
        p = round(np.mean(precs), 3)
        r = round(np.mean(recalls), 3)
        f = round(np.mean(fscores), 3)
        print(rouge_type, 'p:', p, 'r:', r, 'f:', f)

        

rouge_types = ['rouge-1', 'rouge-2', 'rouge-l']
results = dict((rouge_type, collections.defaultdict(list))
               for rouge_type in rouge_types)


lowercase = True

for hyp, ref in cluster_summaries:
    if lowercase:
        hyp = hyp.lower()
        ref = ref.lower()

    r1 = ROUGE_N(ref, hyp, n=1)
    r2 = ROUGE_N(ref, hyp, n=2)
    rl = ROUGE_L(ref, hyp)

    for (rouge_type, scores) in zip(rouge_types, [r1, r2, rl]):
        results[rouge_type]['p'].append(scores.precision)
        results[rouge_type]['r'].append(scores.recall)
        results[rouge_type]['f'].append(scores.fscore)

print_mean(results, rouge_types)


rouge-1 p: 0.251 r: 0.394 f: 0.292
rouge-2 p: 0.079 r: 0.132 f: 0.094
rouge-l p: 0.177 r: 0.284 f: 0.208


In [ ]:
# using 5 articles from cluster results
# evaluation_args = {
#     'evaluation_dataset': '/home/chris/projects/aylien/dynamic-ensembles/data/WCEP/test.jsonl',
#     'model': bart_cnndm_model,
#     'tokenizer': bart_tokenizer,
#     'max_length': 60,
#     'num_beams': 1,
#     'max_articles': 5
# }
# rouge-1 p: 0.247 r: 0.385 f: 0.286
# rouge-2 p: 0.076 r: 0.126 f: 0.091
# rouge-l p: 0.174 r: 0.279 f: 0.204




# using 20 articles from cluster results
# evaluation_args = {
#     'evaluation_dataset': '/home/chris/projects/aylien/dynamic-ensembles/data/WCEP/test.jsonl',
#     'model': bart_cnndm_model,
#     'tokenizer': bart_tokenizer,
#     'max_length': 60,
#     'num_beams': 1,
#     'max_articles': 20
# }
# rouge-1 p: 0.251 r: 0.394 f: 0.292
# rouge-2 p: 0.079 r: 0.132 f: 0.094
# rouge-l p: 0.177 r: 0.284 f: 0.208

In [ ]:
import argparse
import collections
import numpy as np
import utils
from newsroom.analyze.rouge import ROUGE_L, ROUGE_N


def print_mean(results, rouge_types):
    for rouge_type in rouge_types:
        precs = results[rouge_type]['p']
        recalls = results[rouge_type]['r']
        fscores = results[rouge_type]['f']
        p = round(np.mean(precs), 3)
        r = round(np.mean(recalls), 3)
        f = round(np.mean(fscores), 3)
        print(rouge_type, 'p:', p, 'r:', r, 'f:', f)


def evaluate(dataset_path, pred_path, start, stop, lowercase=False):

    dataset = utils.read_jsonl(dataset_path)
    predictions = utils.read_jsonl(pred_path)

    rouge_types = ['rouge-1', 'rouge-2', 'rouge-l']
    results = dict((rouge_type, collections.defaultdict(list))
                   for rouge_type in rouge_types)

    for i, cluster in enumerate(dataset):
        if start > -1 and i < start:
            continue
        if stop > -1 and i >= stop:
            break

        prediction = next(predictions)
        assert prediction['cluster_id'] == cluster['id']

        hyp = prediction['summary']
        ref = cluster['summary']

        if lowercase:
            hyp = hyp.lower()
            ref = ref.lower()

        r1 = ROUGE_N(ref, hyp, n=1)
        r2 = ROUGE_N(ref, hyp, n=2)
        rl = ROUGE_L(ref, hyp)

        for (rouge_type, scores) in zip(rouge_types, [r1, r2, rl]):
            results[rouge_type]['p'].append(scores.precision)
            results[rouge_type]['r'].append(scores.recall)
            results[rouge_type]['f'].append(scores.fscore)

        if i % 100 == 0:
            print(i)
            # print_mean(results, rouge_types)

    print('Final Average:')
    print_mean(results, rouge_types)
    return results


# def main(args):
#     results = evaluate(args.dataset, args.preds, args.start, args.stop,
#                        args.lowercase)
#     utils.write_json(results, args.o)


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--dataset')
#     parser.add_argument('--preds')
#     parser.add_argument('--o')
#     parser.add_argument('--start', type=int, default=-1)
#     parser.add_argument('--stop', type=int, default=-1)
#     parser.add_argument('--lowercase', action='store_true')
#     main(parser.parse_args())

In [ ]:
import json
import pickle


def read_lines(path):
    with open(path) as f:
        for line in f:
            yield line


def read_json(path):
    with open(path) as f:
        object = json.loads(f.read())
    return object


def write_json(object, path):
    with open(path, 'w') as f:
        f.write(json.dumps(object))


def read_jsonl(path, load=False, start=0, stop=None):

    def read_jsonl_gen(path):
        with open(path) as f:
            for i, line in enumerate(f):
                if (stop is not None) and (i >= stop):
                    break
                if i >= start:
                    yield json.loads(line)

    data = read_jsonl_gen(path)
    if load:
        data = list(data)
    return data


def write_jsonl(items, path, batch_size=100, override=True):
    if override:
        with open(path, 'w'):
            pass

    batch = []
    for i, x in enumerate(items):
        if i > 0 and i % batch_size == 0:
            with open(path, 'a') as f:
                output = '\n'.join(batch) + '\n'
                f.write(output)
            batch = []
        raw = json.dumps(x)
        batch.append(raw)

    if batch:
        with open(path, 'a') as f:
            output = '\n'.join(batch) + '\n'
            f.write(output)


def load_pkl(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj


def dump_pkl(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj,  f)


def args_to_summarize_settings(args):
    args = vars(args)
    settings = {}
    for k in ['len_type', 'max_len',
              'min_sent_tokens', 'max_sent_tokens',
              'in_titles', 'out_titles']:
        settings[k] = args[k]
    return settings

In [ ]:
# TODO: BUG -- BEAM ITEMS SHOULD NOT BE THE SAME
# cluster_summaries[0][0] == cluster_summaries[0][1]

In [ ]:
predictions[0] == preditions[1]

In [ ]:
cls.test_news_article_1 = 'New Zealand says it has stopped community transmission of Covid-19, ' \
                    'effectively eliminating the virus. With new cases in single figures for several days - one on Sunday ' \
                    '- Prime Minister Jacinda Ardern said the virus was "currently" eliminated. But officials have warned ' \
                    'against complacency, saying it does not mean a total end to new coronavirus cases. ' \
                    'The news comes hours before New Zealand is set to move out of its toughest level of social restrictions. ' \
                    'From Tuesday, some non-essential business, healthcare and education activity will be able to resume. ' \
                    'Most people will still be required to remain at home at all times and avoid all social interactions.'

cls.test_news_article_2 = \
    'But officials have warned against complacency, saying it does not mean a total end to new HIV cases. ' \
    'Most people will still be required to remain at home at all times and avoid all social interactions.' \
    'Germany says it has stopped community transmission of HIV, ' \
    'effectively eliminating the virus. With new cases in single figures for several days - one on Sunday ' \
    '- Prime Minister Angela Merkle said the virus was "currently" eliminated. ' \
    'From Tuesday, some non-essential business, healthcare and education activity will be able to resume. ' \
    'The news comes hours before Germany is set to move out of its toughest level of social restrictions. '
